In [1]:
#import numpy as np
#import pickle
from util import *

states = [1,2,3]
samples, sequence_len = 100000, 100

initial_states = npr.choice(states, samples).reshape(-1, 1)
dataset = initial_states
cur_state = initial_states
decrement_indices = np.zeros( (samples, sequence_len) ) 

def last_consecutive_count(array):
    condition = array == array[-1]
    agg_counts = 0
    for val in reversed(condition):
        if val:
            agg_counts += 1
        else:
            break
    return agg_counts, condition

for t in range(sequence_len - 1):     
    unit_prob = npr.uniform(0,1, samples)
    active_samples = np.where(unit_prob >= 0.5 )[0]

    for sample in active_samples:            
        prev_pattern_len, condition = last_consecutive_count( dataset[ sample] )
        if prev_pattern_len < 5:
            cur_state[sample] += 1
    
        else:
            #print( dataset[sample], prev_pattern_len, condition) 
            decrement_indices[sample, t-prev_pattern_len+1: t+1] = cur_state[sample] 
            cur_state[sample] -= 1
            
    cur_state[cur_state == 0] = 3
    cur_state[cur_state == 4] = 1 # manual modulo 
    
    dataset = np.hstack( [dataset, cur_state])

pickle.dump( (dataset,decrement_indices) , open('len100_decrement5_123.p', 'wb'))
#print(dataset[0:2])
#print(decrement_indices[0:2])

[[3 3 1 2 3 3 1 1 2 2 2 3 1 1 1 1 1 1 3 1 2 3 1 1 1 2 3 3 3 1 1 1 1 2 2 2 2
  3 1 2 3 3 3 1 2 3 1 1 2 3 1 2 3 1 2 2 2 2 2 2 2 2 2 1 2 3 3 1 2 3 3 3 3 1
  2 3 1 2 3 1 1 1 1 2 2 2 2 3 1 1 2 2 2 3 1 1 2 2 3 3]
 [2 2 3 1 1 1 1 2 3 1 1 2 2 2 2 2 2 2 2 1 2 3 3 3 1 2 3 3 3 3 3 3 3 3 2 2 3
  1 2 2 2 3 3 3 1 2 3 1 1 1 1 2 3 3 3 3 3 2 2 2 3 1 2 2 2 2 2 2 2 1 1 1 1 2
  2 3 3 3 1 1 1 1 1 1 3 1 1 1 1 2 2 3 1 1 2 3 3 3 3 3]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   2.  2.  2.  2.  2.  2.  2.  2.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  2.  2.  2.  2.  2.  2.
   2.  0.  0.  0.  0.  0.  0.  0.  3.  3.  3.  3.  3.  3.  3.  3.  0.  0.
   0.  0.  0.  0.  0.  0

In [ ]:
pca = PCA(n_components = 2)
colors = ['blue', 'green', 'red', 'cyan', 'brown', 'grey', 'orange', 'black', 'purple', 'olive']

state_seq_prob = model.predict_proba(dataset_vec[:100*10], lengths[:10])
two_dim_Z = pca.fit_transform(state_seq_prob)  

state_sequences = model.predict( dataset_vec[:100*10], lengths[:10] )
best_pred_at_state = np.argmax(model.emissionprob_[state_sequences], axis = 1 ).reshape(10, 100)

for sample_num in [3, 4, 5 ]:
    plt.gcf().set_size_inches(18.5, 10.5)
    example_sample = dataset_vec[ sample_num*100: (sample_num+1)*100]
    example_sample_index = np.where( example_sample >= -1)[0]
    short_example_sample = example_sample
    
    labels = example_sample

    prev = -1
    for i in range(sequence_len):
        b = i+(sample_num*100)
        number = labels[b%100][0]
        
        #if number != prev:
        plt.text(two_dim_Z[b,0]-0.04, two_dim_Z[b,1], 
                     '%d'%(example_sample[i]), fontsize=20, color = colors[state_sequences[b%100]])#, color = colors[state_sequence[i%100] ])
        
        truth_color = 'black'
        if example_sample[i] != best_pred_at_state[sample_num, i]:
            truth_color = 'red'
        
        plt.text( i/100-0.4, -0.55, 
                     '%d'%(example_sample[i]), fontsize=20, color = truth_color)
        plt.text( i/100-0.4, -0.5, 
                     '%d'%(best_pred_at_state[sample_num, i]), fontsize=20, color = colors[state_sequences[b%100]])#, color = colors[state_sequence[i%100] ])
        
        plt.arrow(two_dim_Z[b,0], two_dim_Z[b,1], 
                  two_dim_Z[b+1,0] - two_dim_Z[b,0], two_dim_Z[b+1,1] - two_dim_Z[b,1], 
                  color = colors[number], alpha= 0.5,
                  shape='full', lw=0, length_includes_head=True, head_width=.02)
        
        prev = number
        
    plt.ylim(-0.6, 0.6)
    plt.xlim(-0.4, 0.6)
    plt.show()
    
#     for i in example_sample_index:
#         plt.text(two_dim_Z[b,0], two_dim_Z[b,1], number, fontsize=30, color = colors[number])#, color = colors[state_sequence[i%100] ])
#         plt.arrow(two_dim_Z[b,0], two_dim_Z[b,1], 
#                   two_dim_Z[b+1,0] - two_dim_Z[b,0], two_dim_Z[b+1,1] - two_dim_Z[b,1], 
#                   color = colors[number], alpha= 0.2,
#                   shape='full', lw=0, length_includes_head=True, head_width=.02)
        
    